# Multimodal RAG Çözümleri için ColPali Fine-Tune Etmek

Öncelikle bu notebook [Merve Noyan](https://huggingface.co/merve)'ın hazırladığı notebookun Türkçe'ye çevirilmiş ve biraz düzenlenmiş hali. Orijinal haline [buradan](https://github.com/merveenoyan/smol-vision/blob/main/Finetune_ColPali.ipynb?s=35) erişebilirsiniz. Kendisine böyle bir kaynak sağladığı için teşekkür ederiz.

ColPali, çift encoder mimarisine dayalı bir belge-metin sorgu benzerlik modelidir. Model, PaliGemma ve Gemma'ya  dayanmaktadır; burada her modelin görüntü ve metin çıktıları ortak bir alana yansıtılır. Benzerlik, yansıtılan embeddingler arasında hesaplanır ve belgeler ile metinler aralarındaki maksimum benzerliğe göre eşleştirilir. Bu yaklaşımın kendisi şu anda belge erişimi için en gelişmiş yöntemdir. Bu not defterinde ColPali'yi fine tune edeceğiz.

Bu model, multi modal RAG (image + text) dahil olmak üzere belge erişim süreçleri oluşturmak istediğiniz her türlü uygulamada kullanılabilir. Normalde belge erişimi için, görüntü açıklayıcı, tablo-markdown okuyucuları ve OCR modelleri içeren PDF ayrıştırıcıları kullanarak karmaşık bir belgeyi yazıya dökersiniz. ColPali benzeri modeller, bu tür kırılgan ve yavaş süreçlere olan ihtiyacı ortadan kaldırır.

![ColPali Mimarisi](https://github.com/tonywu71/colpali-cookbooks/blob/main/assets/architecture/colpali_architecture.jpeg?raw=true)

Bu not defteri, sentetik olarak oluşturulmuş bir veri seti üzerinde ColPali'yi Türkçe belgeler üzerinde fine tune etmek için bir örnektir. Eğer kendi veri setinizi oluşturmak isterseniz bu [blog yazısını](https://danielvanstrien.xyz/posts/post-with-code/colpali/2024-09-23-generate_colpali_dataset.html) okuyabilirsiniz. Tony Wu tarafından ColPali'nin nasıl fine tune edileceğine dair zaten var olan bir not defteri bulunmaktadır, ancak bu not defteri çok daha basit bir örnek göstermekte ve ColPali'nin transformers versiyonunu kullanmaya odaklanmaktadır.

İndirmemiz gereken kütüphaneler: transformers, peft, bitsandbytes (QLoRa için) ve colpali-engine (contrastive loss implementation için)

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git colpali-engine[train] peft bitsandbytes accelerate tensorboardX flash-attn

In [ ]:
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("GPU memory has been cleared")

Önce Hugging Face'e giriş yapacağız çünkü ColPali, Gemma lisansına sahip PaliGemma'ya dayanmaktadır. Bu not defterini çalıştırmadan önce, Gemma lisansı ile korunan bir model reposunda Gemma lisans sözleşmesini kabul ettiğinizden emin olun.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Modeli Yükleme

ColPali'nin fine tune işlemi yaklaşık 48 GB VRAM gerektirecektir ve bu, 40 GB VRAM'e sahip bir A100 ile uyumlu olmayacaktır. Bellek sınırlarını aşmak için, yalnızca bir adaptör eğitebileceğimiz LoRA veya QLoRA tekniklerini uygulayabiliriz veya modeli 4/8 bit gibi daha düşük bir hassasiyette yükleyip bir adaptör eğitebiliriz. Bu ayarları etkinleştirmek için aşağıda bazı boolean değerlerimiz var; eğer daha iyi bir donanıma sahipseniz, aşağıdaki LORA'yı True ve diğerlerini False olarak ayarlamakta serbestsiniz. Benim önerim modifiye etmeden bu halini kulanmanız.

In [ ]:
from transformers import BitsAndBytesConfig
import torch

QLORA_8BIT = False
QLORA_4BIT = True
LORA = False

if QLORA_8BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
    )
elif QLORA_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
else:
    bnb_config = None

In [ ]:
from transformers import ColPaliForRetrieval, ColPaliProcessor
from peft import LoraConfig, get_peft_model

model_name = "vidore/colpali-v1.3-hf"
model = ColPaliForRetrieval.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="cuda:0",
).eval()

if QLORA_8BIT or QLORA_4BIT or LORA:
  lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        init_lora_weights="gaussian"
    )
  lora_config.inference_mode = False
  model = get_peft_model(model, lora_config)

ColPali Engine'den VisualRetrieverCollator kullanacağız.

In [ ]:
processor = ColPaliProcessor.from_pretrained(model_name)

## Dataseti Yükleme

Bu kısımda biz ayrı datasetler oluşturduğumuzdan modeli eğitmek için bunları bir dataset objesinde topladık. Sizin tek bir datasetniz varsa bu kısmı ona göre değiştirebilirsiniz.

In [ ]:
from datasets import DatasetDict, load_dataset, concatenate_datasets

# Birden fazla dataseti yükleme
dataset1= load_dataset("muhammetfatihaktug/bilim_teknik_mini_colpali")["train"]
dataset2 = load_dataset("selimc/tr-textbook-ColPali")["train"]

# Datasetleri birleştirme
ds = concatenate_datasets([
    dataset1,
    dataset2,
])

# train-test e bölme
split_dataset = ds.train_test_split(test_size=0.1)

# train test datasetleri
train_ds = split_dataset["train"]
test_ds = split_dataset["test"]


print(f"Training set size: {len(train_ds)}")
print(f"Test set size: {len(test_ds)}")

ds

Bu veri seti, Türkçe ders kitaplarından ve bilim dergilerinden alınan belgeleri ve bu belgelerle ilişkili olabilecek sorguları içermektedir. Örneklere kısaca göz atalım.

In [ ]:
idx = 100
print(train_ds[idx]["broad_topical_query"])
print(train_ds[idx]["specific_detail_query"])
print(train_ds[idx]["visual_element_query"])
display(train_ds[idx]["image"])

Bu veri setinden, dökümanları ve sorguları oluşturmak için aşağıdaki sütunlara sahip olacağız:

- `image` (görsel) dökümanlarımızı içerir.

Her görsel için eğitim verisinde 3 ayrı satırımız olacak ve her biri farklı sorgu türü için olacak. Bu şekilde verimizi üçe katlayacak ve farklı sorgu türlerini tanıtacağız.
- `broad_topical_query`
- `specific_detail_query`
- `visual_element_query`

In [ ]:
def collate_fn(examples):
    texts = []
    images = []

    for example in examples:
        query_types = ["broad_topical_query", "specific_detail_query", "visual_element_query"]
         
        # Her bir resimde 3 query tipi için ayrı bir row oluşturuyoruz
        for query_type in query_types:
            if example[query_type] is not None:  # Eğer query boş değilse
                texts.append(example[query_type])
                images.append(example["image"].convert("RGB"))

    batch_images = processor(images=images, return_tensors="pt").to(model.device)
    batch_queries = processor(text=texts, return_tensors="pt").to(model.device)
    return (batch_queries, batch_images)

## Eğiticiyi Oluşturma

Eğitici, ColBERT (contrastive hard-margin loss) kullanır. Bu kayıp fonksiyonu ColPali engine'de uygulanmıştır ve toplu (batch) belge ve sorgu embedding vektörlerini bekler, yani temel olarak belgeleri ve sorguları ayrı ayrı işlememiz, ardından bunları modele ayrı ayrı geçirmemiz ve sonra kayıp hesaplamasına göndermemiz gerekir.

Not: Özel bir kayıp fonksiyonu tanımladığımız için, bunu modele aktarabilmek amacıyla transformers Trainer sınıfını alt sınıf (subclass) olarak kullanmamız gerekiyor.

In [ ]:
import torch
from transformers import Trainer


class ContrastiveTrainer(Trainer):
    def __init__(self, loss_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_func = loss_func

    def compute_loss(self, model, inputs, num_items_in_batch=4, return_outputs=False):
        query_inputs, doc_inputs = inputs
        query_outputs = model(**query_inputs)
        doc_outputs = model(**doc_inputs)
        loss = self.loss_func(query_outputs.embeddings, doc_outputs.embeddings)
        return (loss, (query_outputs, doc_outputs)) if return_outputs else loss

    def prediction_step(self, model, inputs):
        query_inputs, doc_inputs = inputs
        with torch.no_grad():
            query_outputs = model(**query_inputs)
            doc_outputs = model(**doc_inputs)

            loss = self.loss_func(query_outputs.embeddings, doc_outputs.embeddings)
            return loss, None, None

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./turkish-colpali",
    num_train_epochs=1,
    per_device_train_batch_size=2, # per device a 4, gradient accumulation a 4 dediğimizde 80 GB A100 CUDA memory error verdi, o sebeple 2 8 olarak düzenledim.
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    logging_steps=20,
    warmup_steps=100,
    learning_rate=5e-5,
    save_total_limit=1,
    report_to="tensorboard",
    dataloader_pin_memory=False
)

In [ ]:
from colpali_engine.loss import ColbertPairwiseCELoss

trainer = ContrastiveTrainer(
    model=model,
    train_dataset=train_ds,
    args=training_args,
    loss_func=ColbertPairwiseCELoss(),
    data_collator=collate_fn
)

trainer.args.remove_unused_columns = False


In [ ]:
trainer.train()

## Fine-tune edilen modeli yükleme ve test etme

Fine-tune edilmiş modelimizi test edelim. Basitçe metin-görsel çiftlerini modele girerek test edebilir ve gerçekten birbirine ait olan çiftler için skorları kontrol edebilirsiniz. Aynı zamanda alakasız olanların skorlarını da inceleyebilirsiniz (yani eşleşen çiftlerin skorları dışındaki tüm skorlar).

In [ ]:
print(test_ds[0]["specific_detail_query"])
display(test_ds[0]["image"])
print(test_ds[1]["specific_detail_query"])
display(test_ds[1]["image"])
print(test_ds[2]["specific_detail_query"])
display(test_ds[2]["image"])

In [ ]:
images = [test_ds[0]["image"], test_ds[1]["image"], test_ds[2]["image"]]
texts = [test_ds[0]["specific_detail_query"], test_ds[1]["specific_detail_query"], test_ds[2]["specific_detail_query"]]

# process
batch_images = processor(images=images).to(model.device)
batch_queries = processor(text=texts).to(model.device)

# infer
with torch.no_grad():
    image_embeddings = model(**batch_images).embeddings
    query_embeddings = model(**batch_queries).embeddings

# metin-görsel eşleştirmelerini puanlama
scores = processor.score_retrieval(query_embeddings, image_embeddings)

Eşleşen metin-görsel skorları aşağıdaki skor matriksinin köşegeninde yer alıyor, gördüğünüz gibi doğru şekilde eşleşmişler.

In [ ]:
scores

Fine-tune edilmiş modelimizi bu şekilde kendi HuggingFace repoma pushladım.

In [ ]:
trainer.push_to_hub("selimc/turkish-colpali")